In [1]:
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.core.protobuf import saver_pb2
import import_ipynb
import driving_data_new
import model

Instructions for updating:
non-resource variables are not supported in the long term
importing Jupyter notebook from driving_data_new.ipynb
importing Jupyter notebook from model.ipynb
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [2]:
LOGDIR = './save'

In [3]:
sess = tf.InteractiveSession()

In [4]:
L2NormConst = 0.001

In [5]:
train_vars = tf.trainable_variables()

In [6]:
loss = tf.reduce_mean(tf.square(tf.subtract(model.y_, model.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.global_variables_initializer())

In [7]:
# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)

<tf.Tensor 'loss:0' shape=() dtype=string>

In [8]:
# merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [9]:
saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V2)

In [10]:
# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

In [11]:
epochs = 20
batch_size = 100

In [12]:
# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(driving_data_new.num_images / batch_size)):
        xs, ys = driving_data_new.LoadTrainBatch(batch_size)
        train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})
        if i % 10 == 0:
            xs, ys = driving_data_new.LoadValBatch(batch_size)
            loss_value = loss.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * driving_data_new.num_images / batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
print("Model saved in file: %s" % filename)


Epoch: 0, Step: 0, Loss: 6.29895
Epoch: 0, Step: 10, Loss: 6.37607
Epoch: 0, Step: 20, Loss: 6.44906
Epoch: 0, Step: 30, Loss: 6.25885
Epoch: 0, Step: 40, Loss: 6.18236
Epoch: 0, Step: 50, Loss: 6.05436
Epoch: 0, Step: 60, Loss: 6.04507
Epoch: 0, Step: 70, Loss: 5.77935
Epoch: 0, Step: 80, Loss: 5.83152
Epoch: 0, Step: 90, Loss: 5.97141
Epoch: 0, Step: 100, Loss: 5.74603
Epoch: 0, Step: 110, Loss: 5.53378
Epoch: 0, Step: 120, Loss: 5.57288
Epoch: 0, Step: 130, Loss: 5.42585
Epoch: 0, Step: 140, Loss: 5.3072
Epoch: 0, Step: 150, Loss: 5.51574
Epoch: 0, Step: 160, Loss: 5.43737
Epoch: 0, Step: 170, Loss: 5.25247
Epoch: 0, Step: 180, Loss: 5.23586
Epoch: 0, Step: 190, Loss: 5.18608
Epoch: 0, Step: 200, Loss: 4.88373
Epoch: 0, Step: 210, Loss: 5.09755
Epoch: 0, Step: 220, Loss: 5.32081
Epoch: 0, Step: 230, Loss: 5.00313
Epoch: 0, Step: 240, Loss: 4.81133
Epoch: 0, Step: 250, Loss: 4.93885
Epoch: 0, Step: 260, Loss: 4.59636
Epoch: 0, Step: 270, Loss: 4.82327
Epoch: 0, Step: 280, Loss: 4.533

In [13]:
print("Run the command line:\n" \
      "--> tensorboard --logdir=./logs " \
      "\nThen open http://0.0.0.0:6006/ into your web browser")


Run the command line:
--> tensorboard --logdir=./logs 
Then open http://0.0.0.0:6006/ into your web browser
